# Exercise 4.10 TASK (Part 1)

### Importing

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Importing Data
path = r'C:\Users\nmiqu\Desktop\DATA\CareerFoundry\Data Analytics Immersion\April-2024 Instacart Baket Analysis'
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'cust_ords_prods.pkl'))

In [3]:
list(df)

['product_id',
 'product_name',
 'aisle_id',
 'department_id',
 'prices',
 'order_id',
 'user_id',
 'order_number',
 'orders_day_of_week',
 'hour_order_is_made',
 'days_since_prior_order',
 'add_to_cart_order',
 'reordered',
 'price_range',
 'busiest_day',
 'busiest days',
 'Busiest hours',
 'max_order',
 'loyalty_flag',
 'mean_spent',
 'spender_type',
 'frequency_of_order',
 'frequency_type',
 'gender',
 'state',
 'age',
 'date_joined',
 'n_dependants',
 'fam_status',
 'income',
 '_merge']

### Step 2
The information this Dataset contains does not include any personal details that could identify anyone. First and Last name columns have been already dropped, still, not even these would identify a particular person.

### Step 3

In [4]:
# Creating 'region' column
result = []
for value in df['state']:
    if value in ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']:
        result.append('Northeast')
    elif value in ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']:
        result.append('Midwest')
    elif value in ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']:
        result.append('South')
    elif value in ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']:
        result.append('West')

In [5]:
df['region'] = result

In [6]:
df['region'].value_counts(dropna = False)

region
South        10801982
West          8300617
Midwest       7604016
Northeast     5728444
Name: count, dtype: int64

In [7]:
# Creating crosstab to explore spending habits geographycally
crosstab = pd.crosstab(df['region'], df['spender_type'])

In [8]:
crosstab.to_clipboard()

This crosstabs lets us observe some useful information about customers. The highest lowspender operations are located in the South Region, but so is the highest number of Low spenders, this indicate that most customers are located in the South. If we transform the information to percentages, which gives us a more clear view, High spenders represent in each region a 2.05% in Midwest, 1.89% in Northeast, 1.94% in South and a 1.93% in the West.
The distribution is similar in all regions

### Step 4

In [9]:
df.loc[(df['max_order'] <= 5), 'activity_flag'] = 'Low activity'

In [10]:
df.loc[(df['max_order'] > 5), 'activity_flag'] = 'High activity'

In [11]:
df['activity_flag'].value_counts()

activity_flag
High activity    30199644
Low activity      2235415
Name: count, dtype: int64

In [12]:
df_exclusion = df[df['activity_flag'] == 'High activity']

In [13]:
df_exclusion['activity_flag'].value_counts()

activity_flag
High activity    30199644
Name: count, dtype: int64

In [14]:
df_exclusion.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'high_activity_df.pkl'))

### Step 5

I found that I had to import the dataframe again because otherwise, trying the code for this step, python would return  'SettingWithCopyWarning from pandas.' error because it identified it as a slice of the dataframe. When I re-imported it, it worked.

In [2]:
path = r'C:\Users\nmiqu\Desktop\DATA\CareerFoundry\Data Analytics Immersion\April-2024 Instacart Baket Analysis'
df_exclusion = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'high_activity_df.pkl'))

First I will reformat the floats to make it more readable

In [3]:
pd.options.display.float_format = '{:.2f}'.format

In [4]:
df_exclusion[['income', 'age','n_dependants', 'hour_order_is_made']].describe(percentiles = [.33, .66])

,income,age,n_dependants,hour_order_is_made
count,30199644.00,30199644.00,30199644.00,30199644.00
mean,99767.86,49.47,1.50,13.40
std,43181.48,18.49,1.12,4.25
min,25903.00,18.00,0.00,0.00
33%,76561.00,39.00,1.00,11.00
50%,96820.00,49.00,2.00,13.00
66%,114351.00,60.00,2.00,15.00
max,593901.00,81.00,3.00,23.00


I started flaging the income but soon encountered an error because a Nan value, I fixed it by dropping that row and kept on going with the rest of the flags:

In [5]:
# Flag by INCOME
result_2 = []
for value in df_exclusion['income']:
    if value <= 76099:
        result_2.append('low income')
    if (value >= 76100) & (value < 114142):
        result_2.append('mid income')
    if value >= 114142:
        result_2.append('high income')

In [6]:
df_exclusion['income_type'] = result_2

In [7]:
df_exclusion.loc[df_exclusion['income'].isna()]

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,hour_order_is_made,...,state,age,date_joined,n_dependants,fam_status,income,_merge,region,activity_flag,income_type


In [8]:
df_exclusion.dropna(subset = ['income'], inplace = True)

In [9]:
df_exclusion['income_type'] = result_2

In [10]:
df_exclusion['income_type'].value_counts()

income_type
high income    10316038
mid income     10014264
low income      9869342
Name: count, dtype: int64

In [11]:
# Flag by AGE
result_3 = []
for value in df_exclusion['age']:
    if value < 30:
       result_3.append('Young Adult')
    if (value >= 30) & (value < 60):
       result_3.append('Adult')
    if value >= 60:
       result_3.append('Elder')

In [12]:
df_exclusion['age_group'] = result_3

In [13]:
df_exclusion['age_group'].value_counts()

age_group
Adult          14210113
Elder          10315231
Young Adult     5674300
Name: count, dtype: int64

In [14]:
# Flag by HOUR_ORDER_IS_MADE
result_5 = []
for value in df_exclusion['hour_order_is_made']:
    if value in [4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
        result_5.append('Early Bird')
    if value in [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3]:
        result_5.append('Night Owl')

In [15]:
df_exclusion['schedule'] = result_5

In [16]:
df_exclusion['schedule'].value_counts()

schedule
Early Bird    15245418
Night Owl     14954226
Name: count, dtype: int64

In [17]:
# Flag by N_DEPENDANTS
result_6 = []
for value in df_exclusion['n_dependants']:
    if value == 0:
        result_6.append('Non parent')
    if value in [1, 2, 3]:
        result_6.append('Parent')

In [18]:
df_exclusion['parental_status'] = result_6

In [19]:
df_exclusion['parental_status'].value_counts()

parental_status
Parent        22653114
Non parent     7546530
Name: count, dtype: int64

Now I will create more complex flags combining the flags I already just created

In [20]:
# Combined flag AGE and N_DEPENDANTS
df_exclusion.loc[(df_exclusion['age_group'] == 'Young Adult') & (df_exclusion['parental_status'] == 'Parent'), 'family_type'] = 'Young parent'
df_exclusion.loc[(df_exclusion['age_group'] == 'Young Adult') & (df_exclusion['parental_status'] == 'Non parent'), 'family_type'] = 'Single young adult'
df_exclusion.loc[(df_exclusion['age_group'] == 'Adult') & (df_exclusion['parental_status'] == 'Parent'), 'family_type'] = 'Middle age parent'
df_exclusion.loc[(df_exclusion['age_group'] == 'Adult') & (df_exclusion['parental_status'] == 'Non parent'), 'family_type'] = 'Single middle age'
df_exclusion.loc[(df_exclusion['age_group'] == 'Elder') & (df_exclusion['parental_status'] == 'Parent'), 'family_type'] = 'Senior parent'
df_exclusion.loc[(df_exclusion['age_group'] == 'Elder') & (df_exclusion['parental_status'] == 'Non parent'), 'family_type'] = 'Single senior'

In [21]:
df_exclusion['family_type'].value_counts()

family_type
Middle age parent     10655458
Senior parent          7734880
Young parent           4262776
Single middle age      3554655
Single senior          2580351
Single young adult     1411524
Name: count, dtype: int64

### Step 6

The first plot is a simple distribution of the family types:

In [ ]:
family_type_bar = df_exclusion['family_type'].value_counts().plot.bar()

In [23]:
# Exporting the plot
family_type_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'family_type_bar.png'))

I also looked up on Google how to make more interesting plot types and made the following:

In [24]:
# Family type and income_type
crosstab_1 = pd.crosstab(df_exclusion['family_type'], df_exclusion['income_type'], dropna = False)

In [ ]:
income_by_family_type = crosstab_1.plot(kind = 'bar', stacked = True)

In [26]:
income_by_family_type.figure.savefig(os.path.join(path, '04 Analysis', 'Visualizations', 'income_by_family_type.png'))

In [27]:
# Region and family type
crosstab_2 = pd.crosstab(df_exclusion['region'], df_exclusion['family_type'], dropna = False)

In [ ]:
family_type_by_region = crosstab_2.plot(kind = 'bar', stacked = True)
family_type_by_region.legend(loc = 'center right', bbox_to_anchor = (1.5,1))
family_type_by_region 

In [29]:
family_type_by_region.figure.savefig(os.path.join(path, '04 Analysis', 'Visualizations', 'family_type_by_ region.png'))

### Step 7

In [30]:
# Frequency of order by family type
df_exclusion.groupby('family_type').agg({'frequency_of_order' : ['max', 'min', 'mean']})

frequency_of_order          
                                  max  min mean
family_type                                    
Middle age parent               30.00 0.00 9.26
Senior parent                   30.00 0.00 9.27
Single middle age               30.00 0.00 9.21
Single senior                   30.00 0.00 9.18
Single young adult              30.00 0.00 9.31
Young parent                    30.00 0.00 9.22

In [31]:
# Spending by family type
df_exclusion.groupby('family_type').agg({'mean_spent' : ['max', 'min', 'mean']})

mean_spent           
                          max  min  mean
family_type                             
Middle age parent     6389.51 1.00 11.77
Senior parent         3695.99 1.26 11.68
Single middle age    15006.94 1.10 12.25
Single senior        14042.51 1.31 11.60
Single young adult    8794.88 2.60 12.53
Young parent         10209.99 1.90 11.98

### Step 8

 Based on the information from the previous charts we can extract some useful information  
 - All family types have a similar frequency for orders, with a mean around 9 days.
 - Those who spend more money on groceries are Single middle age, and Single young adult.
 

In [50]:
crosstab_3 = pd.crosstab(df_exclusion['family_type'], df_exclusion['department_id'], normalize = 'index') * 100

In [51]:
crosstab_3

department_id,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,10.00,...,12.00,13.00,14.00,15.00,16.00,17.00,18.00,19.00,20.00,21.00
family_type,,,,,,,,,,,,,,,,,,,,,
Middle age parent,6.80,0.11,3.61,29.36,0.46,0.82,8.30,0.29,2.65,0.11,...,2.18,5.75,2.16,3.25,16.75,2.26,1.34,8.98,3.22,0.21
Senior parent,6.85,0.11,3.62,29.39,0.45,0.81,8.37,0.30,2.64,0.11,...,2.20,5.76,2.16,3.27,16.67,2.29,1.32,8.90,3.24,0.21
Single middle age,6.80,0.12,3.66,29.47,0.51,0.83,8.34,0.30,2.59,0.11,...,2.11,5.70,2.15,3.26,16.78,2.19,1.34,8.90,3.28,0.21
Single senior,6.93,0.11,3.62,29.42,0.44,0.82,8.36,0.30,2.65,0.10,...,2.17,5.72,2.14,3.17,16.77,2.19,1.30,8.94,3.28,0.19
Single young adult,6.64,0.11,3.61,29.43,0.43,0.86,8.33,0.31,2.69,0.10,...,2.14,5.73,2.16,3.28,16.68,2.24,1.43,8.99,3.25,0.22
Young parent,6.88,0.12,3.62,29.19,0.49,0.83,8.18,0.31,2.66,0.11,...,2.17,5.73,2.21,3.29,16.86,2.22,1.32,8.98,3.24,0.21


This tables shows us which departments are most popular in each family type, it is represented in percentiles. All family types have similar food choices so I would suggest that the preferences are global and the family type does not affect these.

In [49]:
crosstab_4 = pd.crosstab(df_exclusion['region'], df_exclusion['department_id'], normalize = 'index') * 100

In [1]:
crosstab_4.to_clipboard()

NameError: name 'crosstab_4' is not defined

This tables shows us which departments are most popular in each region, it is represented in percentiles. Numbers are pretty simmilar between all regions so I would suggest that the diet of our costumers is similar across all regions.

In [ ]:
df_exclusion.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_exclusion.pkl'))